In [ ]:
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
from sklearn.linear_model import LinearRegression

In [ ]:
df = pd.read_csv('2012_2020_18-fixed-3.csv', index_col=0).sort_index()

In [ ]:
df

In [ ]:
vikane = df.loc[df['localityNo'] == 13704]   

In [ ]:
vikane

In [ ]:
vikane.loc[vikane['year'] == 2018]

In [ ]:
vikane = 13704

localityNos = [ vikane ]
#[ 13704, 11756, 12982 ]

#localityNos = np.unique(df['localityNo'].to_numpy())[1:100]

#localityNos = [11756, 11754, 11667, 11652, 10080, 14435, 13965, 11738, 10086, 39577, 10309, 13704]

# Calculate growing periods

periods = []

for localityNo in localityNos:
    vikane = df.loc[df['localityNo'] == localityNo]    

    lastYearWeek = { 'year': None, 'week': None, 'yearweek': None }
    currentPeriod = { 'localityNo': localityNo, 'startYear': None, 'startWeek': None, 'endYear': None, 'endWeek': None }

    counter = 0

    for index, row in vikane.iterrows():
        counter = counter + 1
        currentYearWeek = row['year'] * 52 + row['week']

        if not lastYearWeek['yearweek']:
            currentPeriod['startYear'] = row['year']
            currentPeriod['startWeek'] = row['week']
        elif lastYearWeek['yearweek'] and not (lastYearWeek['yearweek'] == currentYearWeek - 1 or lastYearWeek['yearweek'] == currentYearWeek - 2):
            currentPeriod['endYear'] = lastYearWeek['year']
            currentPeriod['endWeek'] = lastYearWeek['week']

            periods.append(copy.copy(currentPeriod))

            currentPeriod['startYear'] = row['year']
            currentPeriod['startWeek'] = row['week']
            currentPeriod['endYear'] = None
            currentPeriod['endWeek'] = None
        elif counter == len(vikane):
            currentPeriod['endYear'] = row['year']
            currentPeriod['endWeek'] = row['week']

            periods.append(copy.copy(currentPeriod))

        lastYearWeek['year'] = row['year']
        lastYearWeek['week'] = row['week']
        lastYearWeek['yearweek'] = currentYearWeek

for period in periods:
    print(period)

In [ ]:
allTreatmentInfo = []

#periods = [ {'localityNo': 10090, 'startYear': 2016, 'startWeek': 1, 'endYear': 2016, 'endWeek': 35} ]

for period in periods:
    localityNo = period['localityNo']
    startYearWeek = period['startYear'] * 52 + period['startWeek']
    endYearWeek = period['endYear'] * 52 + period['endWeek']

    treatmentInfo = []
    
    lastTreatmentYearWeek = None
    lastRow = None
    row = None
    lastLevelsBeforeTreatment = None
    lastLevelsAfterTreatment = None
    
    vikane = df.loc[df['localityNo'] == localityNo]
    
    counter = 0

    for index, nextRow in vikane.iterrows():
        counter = counter + 1
        
        if lastRow is not None:
            currentYearWeek = row['year'] * 52 + row['week']
            
            if currentYearWeek < startYearWeek or currentYearWeek > endYearWeek:
                lastRow = row
                row = nextRow
            
                continue
                
            if row['hasMechanicalRemoval'] or row['hasBathTreatment'] or (counter == len(vikane) and (nextRow['hasMechanicalRemoval'] or nextRow['hasBathTreatment'])):
                beforeRow = lastRow if lastRow['avgAdultFemaleLice'] > row['avgAdultFemaleLice'] else row
                afterRow = nextRow if lastRow['avgAdultFemaleLice'] < row['avgAdultFemaleLice'] else row

                currentLevelsBeforeTreatment = { 'avgAdultFemaleLice': beforeRow['avgAdultFemaleLice'], 'avgMobileLice': beforeRow['avgMobileLice'], 'avgStationaryLice': beforeRow['avgStationaryLice'] }
                currentLevelsAfterTreatment = { 'avgAdultFemaleLice': afterRow['avgAdultFemaleLice'], 'avgMobileLice': afterRow['avgMobileLice'], 'avgStationaryLice': afterRow['avgStationaryLice'] }

                treatmentYearWeek = afterRow['year'] * 52 + afterRow['week']

                if lastTreatmentYearWeek:    
                    weeks = treatmentYearWeek - lastTreatmentYearWeek
                    treatmentInfo.append(copy.copy({ 'hasMechanicalRemoval': row['hasMechanicalRemoval'] or nextRow['hasMechanicalRemoval'], 'hasBathTreatment': row['hasBathTreatment'] or nextRow['hasBathTreatment'], 'lastLevelsBeforeTreatment': lastLevelsBeforeTreatment, 'lastLevelsAfterTreatment': lastLevelsAfterTreatment, 'currentLevelsBeforeTreatment': currentLevelsBeforeTreatment, 'currentLevelsAfterTreatment': currentLevelsAfterTreatment, 'weeks': weeks }))

                lastTreatmentYearWeek = treatmentYearWeek

                lastLevelsBeforeTreatment = currentLevelsBeforeTreatment
                lastLevelsAfterTreatment = currentLevelsAfterTreatment

            
            lastRow = row
            row = nextRow
        elif row is not None:
            lastRow = row 
            row = nextRow
        else:
            row = nextRow
                
    allTreatmentInfo = allTreatmentInfo + treatmentInfo
    
print(len(allTreatmentInfo))

In [ ]:
allTreatmentInfo[0]

In [ ]:
mechanicalTreatmentInfos = [ info for info in allTreatmentInfo if info['hasMechanicalRemoval'] ]
bathTreatmentInfos = [ info for info in allTreatmentInfo if info['hasBathTreatment'] ]

In [ ]:
currentTreatmentInfo = mechanicalTreatmentInfos

beforeTreatmentAdultFemale = np.array([ info['lastLevelsBeforeTreatment']['avgAdultFemaleLice'] for info in currentTreatmentInfo ])
beforeTreatmentMobile = np.array([ info['lastLevelsBeforeTreatment']['avgMobileLice'] for info in currentTreatmentInfo ])
beforeTreatmentStationary = np.array([ info['lastLevelsBeforeTreatment']['avgStationaryLice'] for info in currentTreatmentInfo ])

afterTreatmentAdultFemale = np.array([ info['lastLevelsAfterTreatment']['avgAdultFemaleLice'] for info in currentTreatmentInfo ])
afterTreatmentMobile = np.array([ info['lastLevelsAfterTreatment']['avgMobileLice'] for info in currentTreatmentInfo ])
afterTreatmentStationary = np.array([ info['lastLevelsAfterTreatment']['avgStationaryLice'] for info in currentTreatmentInfo ])

allTreatmentWeeks = np.array([ info['weeks'] for info in currentTreatmentInfo ])
allMechanicalTreatments = np.array([ info['hasMechanicalRemoval'] for info in currentTreatmentInfo ])
allBathTreatments = np.array([ info['hasBathTreatment'] for info in currentTreatmentInfo ])

treatmentEffectAdultFemale = beforeTreatmentAdultFemale - afterTreatmentAdultFemale
treatmentEffectMobile = beforeTreatmentMobile - afterTreatmentMobile
treatmentEffectStationary = beforeTreatmentStationary - afterTreatmentStationary

plt.plot(treatmentEffectAdultFemale, allTreatmentWeeks, 'p')

In [ ]:
print(np.mean(beforeTreatmentAdultFemale), np.mean(afterTreatmentAdultFemale), 1 - (np.mean(afterTreatmentAdultFemale) / np.mean(beforeTreatmentAdultFemale)))
print(np.mean(beforeTreatmentMobile), np.mean(afterTreatmentMobile), 1 - (np.mean(afterTreatmentMobile) / np.mean(beforeTreatmentMobile)))
print(np.mean(beforeTreatmentStationary), np.mean(afterTreatmentStationary), 1 - (np.mean(afterTreatmentStationary) / np.mean(beforeTreatmentStationary)))



In [ ]:
print(np.mean(beforeTreatmentAdultFemale), np.mean(afterTreatmentAdultFemale), 1 - (np.mean(afterTreatmentAdultFemale) / np.mean(beforeTreatmentAdultFemale)))
print(np.mean(beforeTreatmentMobile), np.mean(afterTreatmentMobile), 1 - (np.mean(afterTreatmentMobile) / np.mean(beforeTreatmentMobile)))
print(np.mean(beforeTreatmentStationary), np.mean(afterTreatmentStationary), 1 - (np.mean(afterTreatmentStationary) / np.mean(beforeTreatmentStationary)))



In [ ]:
print(np.mean(allMechanicalTreatments), np.mean(allBathTreatments))

In [ ]:
np.mean(allTreatmentWeeks)

In [ ]:
np.mean(allTreatmentWeeks)

In [ ]:
subset = np.where((treatmentEffectAdultFemale < 0.5) &  (treatmentEffectAdultFemale > -0.25))

x = treatmentEffectAdultFemale[subset]
Y = allTreatmentWeeks[subset]

X = x.reshape(-1, 1)

plt.plot(X, Y, 'p')

reg = LinearRegression().fit(X, Y)

print(reg.intercept_, reg.coef_)
print(reg.score(X, Y))

predTreatmentWeeks = reg.predict(X)

plt.plot(X, predTreatmentWeeks, '-')

In [ ]:
subset = np.where((afterTreatmentStationary < 0.5))

x = afterTreatmentStationary[subset]
Y = allTreatmentWeeks[subset]

X = x.reshape(-1, 1)

plt.plot(X, Y, 'p')

reg = LinearRegression().fit(X, Y)

print(reg.intercept_, reg.coef_)
print(reg.score(X, Y))

predTreatmentWeeks = reg.predict(X)

plt.plot(X, predTreatmentWeeks, '-')

In [ ]:
subset = np.where((afterTreatmentMobile < 0.5))

x = afterTreatmentMobile[subset]
Y = allTreatmentWeeks[subset]

X = x.reshape(-1, 1)

plt.plot(X, Y, 'p')

reg = LinearRegression().fit(X, Y)

print(reg.intercept_, reg.coef_)
print(reg.score(X, Y))

predTreatmentWeeks = reg.predict(X)

plt.plot(X, predTreatmentWeeks, '-')

In [ ]:
subset = np.where((afterTreatmentAdultFemale < 0.5))

x = afterTreatmentAdultFemale[subset]
Y = allTreatmentWeeks[subset]

X = x.reshape(-1, 1)

plt.plot(X, Y, 'p')

reg = LinearRegression().fit(X, Y)

print(reg.intercept_, reg.coef_)
print(reg.score(X, Y))

predTreatmentWeeks = reg.predict(X)

plt.plot(X, predTreatmentWeeks, '-')

In [ ]:
print(np.mean(beforeTreatmentAdultFemale), np.mean(afterTreatmentAdultFemale), 1 - (np.mean(afterTreatmentAdultFemale) / np.mean(beforeTreatmentAdultFemale)))
print(np.mean(beforeTreatmentMobile), np.mean(afterTreatmentMobile), 1 - (np.mean(afterTreatmentMobile) / np.mean(beforeTreatmentMobile)))
print(np.mean(beforeTreatmentStationary), np.mean(afterTreatmentStationary), 1 - (np.mean(afterTreatmentStationary) / np.mean(beforeTreatmentStationary)))



In [ ]:
np.mean(allTreatmentWeeks) * 7